### **Instalando e importando as bibliotecas necessárias**

In [397]:
%pip install PyOpenGL
%pip install glfw
%pip install pyglm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [398]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm

### **Inicializando a janela**

In [399]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Avião em voo", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### **Shaders**
Defininindo os programas Vertex e Fragment Shaders e requisitando slot para a GPU para eles. Após isso, associamos nosso código-fonte aos slots associados.

In [400]:
vertex_code = """
    #version 330 core
    layout (location = 0) in vec3 position;
    uniform mat4 mat_transformation;
    
    void main() {
        gl_Position = mat_transformation * vec4(position, 1.0);
    }
"""

fragment_code = """
    #version 330 core
    uniform vec4 color;
    out vec4 FragColor;
    
    void main() {
        FragColor = color;
    }
"""

program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

Compilando nossos programas e associando ao programa principal

In [401]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem e Preparação de Dados para a GPU

Build do programa tornando ele default

In [402]:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
glUseProgram(program)

Modelagem dos objetos em cena

In [403]:
vertices = np.array([
    # Fuselagem (retangular)
    ( 0.75,  0.2,  0.2),  # 0
    (-0.75,  0.2,  0.2),  # 1
    (-0.75, -0.2,  0.2),  # 2 
    ( 0.75, -0.2,  0.2),  # 3
    ( 0.75,  0.2, -0.2),  # 4
    (-0.75,  0.2, -0.2),  # 5
    (-0.75, -0.2, -0.2),  # 6
    ( 0.75, -0.2, -0.2),  # 7

    # Asa direita
    (-0.25,  0.025, 1.2),  # Vértice ponta (meio da face) 8
    (-0.25,  0.025, 0.2),  # Base superior esquerda 9
    (0.25, 0.025, 0.2),  # Base inferior esquerda 10
    (-0.25,  -0.025, 1.2),  # Vértice ponta (fundo do prisma) 11
    (-0.25,  -0.025, 0.2),  # Base superior esquerda (fundo) 12
    (0.25, -0.025, 0.2),   # Base inferior esquerda (fundo) 13

    # Asa esquerda
    (-0.25,  0.025, -1.2),  # 14
    (-0.25,  0.025, -0.2),  # 15
    (0.25, 0.025, -0.2),  #16
    (-0.25,  -0.025, -1.2),  #17
    (-0.25,  -0.025, -0.2),  #18
    (0.25, -0.025, -0.2),   #19

    # Nariz
    ( 0.95,  0.1,  0.1), # 20 
    ( 0.95,  -0.1,  0.1), # 21
    ( 0.95,  0.1,  -0.1), # 22
    ( 0.95,  -0.1,  -0.1), # 23 (parte branca do nariz até aqui)
    ( 1.2, 0.0, 0.0), # 24 (pontinha vermelha do nariz)

    # Traseira do avião
    ( -1.2, 0.2, 0.0), # 25

    # Barbatana do avião
    ( -0.8, 0.2, 0.025), # 26
    ( -0.8, 0.2, -0.025), # 27
    ( -1.0, 0.2, 0.025), # 28
    ( -1.0, 0.2, -0.025), # 29
    ( -1.0, 0.6, 0.025), # 30
    ( -1.0, 0.6, -0.025), # 31
    ( -0.95, 0.6, 0.025), # 32
    ( -0.95, 0.6, -0.025), # 33

    # Estabilizador Direito
    (-0.95,  0.025, 0.4),  # 34
    (-0.95,  0.025, 0.05),  # 35
    (-0.80, 0.025, 0.15),  # 36
    (-0.95,  -0.025, 0.4),  # 37
    (-0.95,  -0.025, 0.05),  # 38
    (-0.80, -0.025, 0.15),   # 39

    # Estabilizador Esquerdo
    (-0.95,  0.025, -0.4),  # 40
    (-0.95,  0.025, -0.05),  # 41
    (-0.80, 0.025, -0.15),  # 42
    (-0.95,  -0.025, -0.4),  # 43
    (-0.95,  -0.025, -0.05),  # 44
    (-0.80, -0.025, -0.15),   # 45

], dtype=np.float32)

import numpy as np

indices = np.array([
    # Fuselagem (36 índices)
    0, 1, 2,  0, 2, 3,
    4, 6, 5,  4, 7, 6,
    0, 4, 5,  0, 5, 1,
    2, 6, 7,  2, 7, 3,
    0, 3, 7,  0, 7, 4,
    1, 5, 6,  1, 6, 2,

    # Asa Direita (24 índices)
    8, 9, 10,
    11, 13, 12,
    8, 9, 12,  8, 12, 11,
    8, 10, 13,  8, 13, 11,
    9, 10, 13,  9, 13, 12,

    # Asa Direita (24 índices)
    14, 15, 16,
    17, 19, 18,
    14, 15, 18,  14, 18, 17,
    14, 16, 19,  14, 19, 17,
    15, 16, 19,  15, 19, 18,

    # Nariz parte branca (24 índices)
    20, 21, 22,
    21, 22, 23,
    0, 3, 20,  3, 20, 21,
    0, 4, 20,  4, 20, 22,
    4, 7, 22,  7, 22, 23,
    3, 7, 23,  3, 21, 23,

    # Nariz parte vermelha (12 índices)
    20, 22, 24,
    20, 21, 24,
    21, 23, 24,
    22, 23, 24,

    # Traseira do avião (12 índices)
    1, 2, 25,
    2, 5, 25,
    5, 6, 25,
    6, 1, 25,

    # Barbatana (30 índices)
    26, 28, 30,  26, 30, 32,
    27, 29, 31,  27, 31, 33,
    26, 27, 32,  27, 32, 33,
    31, 32, 33,  30, 31, 32,
    28, 29, 30,  29, 30, 31,

    # Estabilizador Direito (24 índices)
    34, 35, 36,
    37, 39, 38,
    34, 35, 38,  34, 38, 37,
    34, 36, 39,  34, 39, 37,
    35, 36, 39,  35, 39, 38,

    # Estabilizador Esquerdo (24 índices)
    40, 41, 42,
    43, 45, 44,
    40, 41, 44,  40, 44, 43,
    40, 42, 45,  40, 45, 43,
    41, 42, 45,  41, 39, 44,

], dtype=np.uint32)


Configuração VAO/VBO/EBO

In [404]:
VAO = glGenVertexArrays(1)
glBindVertexArray(VAO)

VBO = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, VBO)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

EBO = glGenBuffers(1)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO)
glBufferData(GL_ELEMENT_ARRAY_BUFFER, indices.nbytes, indices, GL_STATIC_DRAW)

Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

In [405]:
glVertexAttribPointer(0, 3, GL_FLOAT, False, 12, ctypes.c_void_p(0))
glEnableVertexAttribArray(0)

Definindo cores do projeto (mudar pra cá dps)

### **Capturando eventos de teclado e modificando variáveis para a matriz de transformação**

In [406]:
fator_escala = 0.5

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, angulo, escala
    
    if key == 265: t_y += 0.01 #cima
    if key == 264: t_y -= 0.01 #baixo
    if key == 263: t_x -= 0.01 #esquerda
    if key == 262: t_x += 0.01 #direita

    if key == 65:  angulo += 0.1 # gira para a direita
    if key == 83:  angulo -= 0.1 # gira para a esquerda

    if key == 90: escala += 0.1 * fator_escala # aumenta escala 
    if key == 88: escala -= 0.1 * fator_escala # diminui escala

    if key == 82: 
        t_x = 0
        t_y = 0
        angulo = 0
        escala = 1


    
glfw.set_key_callback(window,key_event)

### **Funções Principais da Janela**

In [407]:
glfw.show_window(window)

In [408]:
import math

angulo = 0
t_x = 0
t_y = 0
escala = 1


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

# Matriz de projeção ortográfica (mapeia coordenadas 3D para a tela)
glEnable(GL_DEPTH_TEST)
glClearColor(0.53, 0.81, 0.92, 1.0)  # Cor de céu

# Matriz de projeção perspectiva
width, height = glfw.get_window_size(window)
proj = glm.perspective(glm.radians(60.0), width/height, 0.1, 100.0)

# Posição da câmera
view = glm.lookAt(
    glm.vec3(2, 2, 2),  # Posição da câmera
    glm.vec3(0, 0, 0),  # Ponto para onde olha
    glm.vec3(0, 1, 0)   # Vetor "para cima"
)

# Enviamos para o shader
loc_transform = glGetUniformLocation(program, "mat_transformation")
loc_color = glGetUniformLocation(program, "color")


# Loop principal
while not glfw.window_should_close(window):
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Animação de rotação
    angulo += 0.0001
    model = glm.mat4(1.0)
    model = glm.rotate(model, angulo, glm.vec3(0, 1, 0))  # Rotaciona em Y
    
    # Configura transformações
    mat_model = glm.mat4(1.0)
    mat_model = glm.translate(mat_model, glm.vec3(t_x, t_y, 0.0))
    mat_model = glm.rotate(mat_model, angulo, glm.vec3(0.0, 0.0, 1.0))
    mat_model = glm.scale(mat_model, glm.vec3(escala, escala, 1.0))
    
    # Matriz final
    mvp = proj * view * model
    
    # Envia para o shader
    glUniformMatrix4fv(loc_transform, 1, GL_FALSE, glm.value_ptr(mvp))
    
    # Desenha as partes do avião com cores diferentes
    # Corpo principal (branco)
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0)
    glDrawElements(GL_TRIANGLES, 36, GL_UNSIGNED_INT, None)

    # Asas (vermelhas)
    glUniform4f(loc_color, 1.0, 0.0, 0.0, 1.0)
    glDrawElements(GL_TRIANGLES, 48, GL_UNSIGNED_INT, ctypes.c_void_p(36 * 4))

    # Nariz
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) # parte branca
    glDrawElements(GL_TRIANGLES, 30, GL_UNSIGNED_INT, ctypes.c_void_p(84 * 4))
    glUniform4f(loc_color, 1.0, 0.0, 0.0, 1.0) # parte vermelha
    glDrawElements(GL_TRIANGLES, 12, GL_UNSIGNED_INT, ctypes.c_void_p(114 * 4))

    # Traseira
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) 
    glDrawElements(GL_TRIANGLES, 12, GL_UNSIGNED_INT, ctypes.c_void_p(126 * 4))

    # Barbatana
    glUniform4f(loc_color, 1.0, 0.0, 0.0, 1.0) 
    glDrawElements(GL_TRIANGLES, 30, GL_UNSIGNED_INT, ctypes.c_void_p(138 * 4))

    # Estabilizador direito
    glUniform4f(loc_color, 1.0, 0.0, 0.0, 1.0) 
    glDrawElements(GL_TRIANGLES, 48, GL_UNSIGNED_INT, ctypes.c_void_p(168 * 4))

    # Descomente as duas linhas abaixo para conseguir ver as arestas
    glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0)  # Cor preta
    glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)    # Modo wireframe
    glDrawElements(GL_TRIANGLES, len(indices), GL_UNSIGNED_INT, None)
    glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)    # Volta ao modo sólido
    
    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()